In [ ]:
# Install a pip package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install git+https://github.com/ngargaud/hive-api.git
!{sys.executable} -m pip install pillow # to show images in the notebook

In [ ]:
import IPython
import json
from PIL import Image

from hive_api import HiveApi

test_ollama=True
api = HiveApi()

# get image from the sandbox
screenshot = await api.get_sandbox_screenshot(filename="/tmp/screenshot.png")
IPython.display.display(screenshot)

# change settings for asr
# print(api.get_api_settings("asr"))
# api.set_asr_task("translate")
# print(api.get_api_settings("asr"))

# change settings for tts
# print(api.get_api_settings("tts"))
# api.set_tts_lang("en")
# api.set_tts_clone(False)

# Generate an audio file with the TTS module then feed it to the ASR module
# In blocking and non blocking modes
#
print("TTS blocking output")
output_tts = api.call_tts("hello there")
job_tts = api.call_tts("hello there, but in a separated thread to let the notebook show the first audio file", wait=False)
IPython.display.display(IPython.display.Audio(output_tts))

print("TTS non blocking output, started before displaying the first output")
output_tts2 = api.fetch_tts_file(job_tts.result().get("value"))
job_asr = api.call_asr(output_tts2, wait=False)
IPython.display.display(IPython.display.Audio(output_tts2))

print("TTS non blocking output, calling asr on the tts output, started before displaying the second output")
print("ASR blocking res:",api.call_asr(output_tts))
print("ASR non blocking res:", job_asr.result())

# api.set_tts_lang("fr-fr")
# api.set_tts_clone(True)
# print(api.get_api_settings("tts"))
# print(api.get_api_settings("voicereco"))
# print(api.get_api_settings("facereco"))

ollama = api.get_client("ollama")
# print(json.dumps(ollama.list(), indent=2))
if test_ollama:
    model = "mistral"
    prompt = "what is 45 * 6?"
    ollama.pull(model)
    ollama.show(model)
    
    sync = ollama.chat(model=model, messages=[{'role': 'user', 'content': prompt}])
    print(json.dumps(sync, indent=2))
    print("-"*30)

    print("Streaming chat")
    stream = ollama.chat(model=model, messages=[{'role': 'user', 'content': prompt}], stream=True)
    for chunk in stream:
        print(chunk['message']['content'], end='', flush=True)
    print("")
    print("-"*30)

    print("Image description with llava")
    ollama.pull('llava:v1.6')
    stream = ollama.generate(model='llava:v1.6', prompt='What is this image', images=["/tmp/screenshot.png"], stream=True)
    for chunk in stream:
        print(chunk['response'], end='', flush=True)